<div >
<img src = "../banner.jpg" />
</div>

<a target="_blank" href="https://colab.research.google.com/github/ignaciomsarmiento/BDML_202501/blob/main/Modulo05/CuadernoModulo05_ArbolesBosques.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Tree-Based Methods

## Motivación: Prediciendo Precios de propiedades


$$
Precio=f(Habitaciones,DCBD)
$$



## CARTs

Cargamos los paquetes

In [ ]:
# install.packages("pacman") #run this line if you use Google Colab

In [ ]:
#packages
require("pacman")
p_load("tidyverse","ggplot2")

### Toy example

In [ ]:
db<-read.csv('https://raw.githubusercontent.com/ignaciomsarmiento/datasets/main/toy_houses.csv')

In [ ]:
dim(db)

In [ ]:
head(db)

In [ ]:
summary(db)

In [ ]:
ggplot(db) +
  geom_point(aes(x=habitaciones,y=DCBD),position=position_jitter(width = .05)) +
  scale_x_continuous(breaks=seq(0,8,1)) +
  theme_classic() +
  xlab("Habitaciones") +
  ylab("Distancia al Centro") +
  theme(legend.position =  "none",
      text=element_text(size=20))

#### Algorithm


-  Datos: $y_{n\times 1}$  y $X_{n\times k}$ 

-  Definiciones

      -  *j* es la variable que parte el espacio 
      - *s* es el punto de partición


-  Definimos los siguientes semiplanos

\begin{align}
R_1(j,s)=\{X|X_j\leq s\} \,\,\, \& \,\,\, R_2(j,s)=\{X|X_j > s\}
\end{align}

-  *El problema*: buscar la variable de partición $X_j$ y el punto $s$ de forma tal que 


\begin{align}
\underset{j,s}{min} \left[ \underset{y_{R_1}}{min}\sum_{x_i\in R_1(j,s)}(y-y_{R_1})^2+ \underset{y_{R_2}}{min}\sum_{x_i\in R_2(j,s)}(y-y_{R_2})^2\right]
\end{align}



#### Algorithm by hand ("artesanal")

1. Iniciemos por DBCD

In [ ]:
MSE_dbcd<-NA

j<-1
for(i in seq(1.25,1.75,0.25)){
    #Region 1
  R1<- db %>% filter(DCBD<=i)
  R1<- R1 %>% mutate(yR1=mean(price))
  MSEr1<- sum((R1$price-R1$yR1)^2)
 #Region 2
  R2<- db %>% filter(DCBD>i)
  R2<- R2 %>% mutate(yR2=mean(price))
  MSEr2<- sum((R2$price-R2$yR2)^2)
  
  MSE_dbcd[j]<-MSEr1+MSEr2
  j<-j+1
}

MSE_dbcd

2. Luego por Habitaciones

In [ ]:
MSE_hab<-NA

for(i in 0:7){
  R1<- db %>% filter(habitaciones<=i)
  R1<- R1 %>% mutate(yR1=mean(price))
  MSEr1<- sum((R1$price-R1$yR1)^2)
  R2<- db %>% filter(habitaciones>i)
  R2<- R2 %>% mutate(yR2=mean(price))
  MSEr2<- sum((R2$price-R2$yR2)^2)
  
  MSE_hab[i+1]<-MSEr1+MSEr2
  
}
MSE_hab

**Mínimo?**

In [ ]:
MSE<-c(MSE_dbcd,MSE_hab)
MSE[which.min(MSE)]
MSE

#### Algorithm with `rpart`

Sin embargo, no construimos el árbol a mano, usamos el paquete `rpart`, que implementa el algoritmo de árboles de decisión para regresión y clasificación. Este paquete permite ajustar modelos de manera rápida y sencilla:

In [ ]:
p_load("rpart")
mytree <- rpart(price~DCBD+habitaciones,data=db)

In [ ]:
mytree

In [ ]:
plot(mytree)
text(mytree)

Para visualizar árboles de decisión de forma mas "bonita", podemos usar el paquete `rpart.plot`, que extiende las capacidades de rpart con funciones gráficas más potentes y estéticas. La función principal es `prp()` qye permite crear gráficos detallados del árbol entrenado. En el siguiente ejemplo, usamos varias opciones para mejorar la presentación del árbol:

In [ ]:
p_load("rpart.plot")

prp(mytree, 
    under = TRUE,              # Muestra información adicional debajo del nodo
    branch.lty = 2,            # Estilo de línea punteada para las ramas
    yesno = 2,                 # Muestra "sí/no" en las bifurcaciones
    faclen = 0,                # Muestra etiquetas completas de factores
    varlen = 15,               # Longitud máxima del nombre de la variable
    tweak = 1.2,               # Ajusta el tamaño del texto
    clip.facs = TRUE,          # Recorta niveles largos de factores
    box.palette = "Greens",    # Paleta de colores para las cajas
    compress = TRUE,           # Comprime el árbol verticalmente
    ycompress = TRUE,          # Comprime también el eje y
    node.fun = function(x, labs, digits, varlen) 
      paste("Precio \n", format(round(mytree$frame$yval, 2), nsmall=0, big.mark=","))  # Personaliza el texto del nodo
)


Recordemos que la forma general del arbol es
$$
f(x_{i};\{R_{j},\gamma_{j}\}_{1}^{J})=\sum_{j=1}^{J}\gamma_{j}I(x\in R_{j})
$$

Entonces en este caso, el precio predicho por el  árbol es 

\begin{align}
\hat{Precio} = 162,754.67 I(DCBD<1.5) + 98,716.14 I(DCBD>=1.5 \& habitaciones>=3) + 73,130.58 I(DCBD>=1.5 \& habitaciones<3) 
\end{align}

### Ames Data Set

Este conjunto de datos contiene información detallada sobre las características de viviendas en Ames, Iowa, y su precio de venta. Incluye más de 80 variables que describen aspectos físicos, de ubicación y de calidad de las propiedades, como el tamaño habitable, el tipo de edificio, el año de construcción, el número de baños, y si tiene garaje o cercado, entre otros.

Es ampliamente utilizado en ciencia de datos y machine learning como un caso realista para problemas de regresión.  
La descripción completa de las variables puede consultarse aquí:  
https://jse.amstat.org/v19n3/decock/DataDocumentation.txt



In [ ]:
p_load("modeldata")

data("ames", package = "modeldata")

In [ ]:
dim(ames)

In [ ]:
head(ames)

In [ ]:
amestree <- rpart(log(Sale_Price) ~  Gr_Liv_Area  + Bldg_Type + Fence + Lot_Area,data=ames, control = list(maxdepth = 2))


In [ ]:

p_load("rpart.plot")
prp(amestree, under = TRUE, branch.lty = 2, yesno = 2, faclen = 0, varlen=15,tweak=1.2,clip.facs= TRUE,box.palette = "Greens",compress=FALSE,ycompress = FALSE,node.fun=function(x, labs, digits, varlen) paste("Precio \n", format(round(exp(amestree$frame$yval), 0), nsmall=0, big.mark=",")))

### Sobreajuste

<div >
<img src = "figures/tree_uba.png" width="300"/>
</div>


- Fijar la profundidad del árbol. (implementado en Caret `method=rpart2`)

- Fijar la mínima cantidad de datos que están contenidos dentro de cada hoja. 

- Cost complexity pruning (implementado en Caret con `method=rpart`)

- `tidymodels` implementa todos

##### Implementación con Caret

In [ ]:
p_load("caret")

In [ ]:
fitControl<-trainControl(method ="cv",
                         number=5)

#####  `method=rpart2`  allows to tune Max Tree Depth

In [ ]:
set.seed(123)
tree_rpart2 <- train(
    log(Sale_Price) ~ Gr_Liv_Area  + Bldg_Type + Fence + Lot_Area,
    data=ames,
    method = "rpart2",
    trControl = fitControl,
    tuneGrid = expand.grid(maxdepth = seq(1,8,1))
)

In [ ]:
tree_rpart2

In [ ]:
prp(tree_rpart2$finalModel, under = TRUE, branch.lty = 2, yesno = 2, faclen = 0, varlen=15,tweak=1.2,clip.facs= TRUE,box.palette = "Greens",compress=FALSE,ycompress = FALSE,node.fun=function(x, labs, digits, varlen) paste("Precio \n", format(round(exp(tree_rpart2$finalModel$frame$yval), 0), nsmall=0, big.mark=",")))

#### Cost complexity Prunning


Cost complexity del árbol  $T$ con $[T]$ nodos terminales del árbol 
\begin{align}
  C_{\alpha}(T)= \sum_{m=1}^{[T]}  \sum_{x_i\in R_m} (y_i-\hat{y}_m)^2 + \alpha [T]
\end{align}


Objetivo: para un dado $\alpha$, encontrar el pruning óptimo que minimice  $C_{\alpha}(T)$

Se logra eliminando sucesivamente las ramas que producen un aumento mínimo en $\sum_{x_i\in R_m} (y_i-\hat{y}_m)^2 $


##### Algoritmo completo

  - Hacemos crecer el árbol

  - Para un dado $\alpha$, aplicamos  *cost complexity pruning* 
    
  - Utilizamos K-fold cross-validation para elegir $\alpha$. 

  
Tenemos entonces una secuencia de subarboles para distintos valores de $\alpha$ 

Elegimos el $\alpha$ y el subárbol que tienen el menor error de predicción.

#####  `method=rpart`  only allows to tune Complexity Parameter

- Can change the length


In [ ]:
set.seed(123)
tree_lenght <- train(
    log(Sale_Price) ~ Gr_Liv_Area  + Bldg_Type + Fence + Lot_Area,
    data=ames,
    method = "rpart",
    trControl = fitControl,
    tuneLength=20
)

In [ ]:
tree_lenght

In [ ]:
prp(tree_lenght$finalModel, under = TRUE, branch.lty = 2, yesno = 2, faclen = 0, varlen=15,tweak=1.2,clip.facs= TRUE,box.palette = "Greens",compress=FALSE,ycompress = FALSE,node.fun=function(x, labs, digits, varlen) paste("Precio \n", format(round(exp(tree_lenght$finalModel$frame$yval), 0), nsmall=0, big.mark=",")))

- Or the grid

In [ ]:
set.seed(123)
tree_grid <- train(
    log(Sale_Price) ~ Gr_Liv_Area  + Bldg_Type + Fence + Lot_Area,
    data=ames,
    method = "rpart",
    trControl = fitControl,
    tuneGrid = expand.grid(cp = seq(0.001707763, 0.001707765, length.out = 100))
)


In [ ]:
tree_grid

More details here: https://topepo.github.io/caret/train-models-by-tag.html#tree-based-model

### Comentarios sobre Árboles


#### Pros: 
  
    - Los árboles son muy fáciles de explicar a las personas (probablemente incluso más fáciles que la regresión lineal)

    - Los árboles se pueden trazar gráficamente y son fácilmente interpretados incluso por no expertos. Variables más importantes en la parte superior



#### Cons:
    
    - Si la estructura es lineal, CART no funciona bien
    
<div >
<img src = "figures/tree_vs_reg.png" />
</div>


    - Los árboles no son muy robustos 
    

## Bagging and Random Forests

### Intuición con código


Una forma de mejorar la estabilidad y precisión de los árboles de decisión es a través del bagging (bootstrap aggregating). La idea central es entrenar múltiples árboles sobre distintas muestras del conjunto de datos —obtenidas con reemplazo (bootstrap)— y luego combinar sus predicciones. Esto reduce la varianza del modelo y mejora su capacidad de generalización.

En este ejemplo, generamos una única muestra bootstrap del dataset original y entrenamos un árbol simple sobre esa muestra:

In [ ]:
set.seed(101010)
sample_ames2 <- sample_frac(ames, size = 1, replace = TRUE) 

Entrenamos un árbol poco profundo (maxdepth = 2) para facilitar la visualización y destacar cómo el modelo aprende patrones sobre una muestra específica del conjunto. 

In [ ]:
amestree2 <- rpart(log(Sale_Price) ~ Gr_Liv_Area + Bldg_Type + Fence + Lot_Area,
                   data = sample_ames2,
                   control = list(maxdepth = 2))

Luego, visualizamos el árbol con `prp()` personalizando las etiquetas de los nodos para mostrar el precio estimado en escala original (deshaciendo el log):

In [ ]:
prp(amestree2, 
    under = TRUE, branch.lty = 2, yesno = 2, faclen = 0, varlen = 15, tweak = 1.2,
    clip.facs = TRUE, box.palette = "Greens", compress = FALSE, ycompress = FALSE,
    node.fun = function(x, labs, digits, varlen) 
      paste("Precio \n", format(round(exp(amestree2$frame$yval), 0), nsmall = 0, big.mark = ","))
)


Comparamos el árbol ajustado sobre la muestra bootstrap (`amestree2`) con el árbol original (`amestree`), entrenado sobre el conjunto de datos original. Esta comparación nos permite visualizar cómo el muestreo con reemplazo introduce variabilidad en la estructura del árbol —una idea clave detrás del bagging.

In [ ]:
prp(amestree, under = TRUE, branch.lty = 2, yesno = 2, faclen = 0, varlen=15,tweak=1.2,clip.facs= TRUE,box.palette = "Greens",compress=FALSE,ycompress = FALSE,node.fun=function(x, labs, digits, varlen) paste("Precio \n", format(round(exp(amestree$frame$yval), 0), nsmall=0, big.mark=",")))

Como vemos formalmente las predicciones de Bagging?

Formalmente, si denotamos los árboles como funciones:

$$
T_{\text{1}}(x) = \sum_{j=1}^{J_1} \gamma_{j}^{(1)} \cdot \mathbf{1}(x \in R_j^{(1)}), \quad
T_{\text{2}}(x) = \sum_{k=1}^{J_2} \gamma_{k}^{(2)} \cdot \mathbf{1}(x \in R_k^{(2)})
$$

donde cada $ R_j^{(1)} $ y $ R_k^{(2)} $ es una región terminal (una hoja) del árbol correspondiente, y $ \gamma 4 es la predicción constante en esa región.

Entonces, si quisiéramos combinar estos dos árboles, la predicción combinada sería simplemente el promedio:

$
\hat{f}(x) = \frac{1}{2} \left( T_{\text{1}}(x) + T_{\text{2}}(x) \right)
$

En términos prácticos, para cualquier observación \( x \), cada árbol le asigna un valor predicho dependiendo de la región hoja en la que cae, y el modelo final sería el promedio de ambos.


### Con `caret` y `ranger`

Podemos entrenar estos modelos  utilizando el paquete `ranger` en combinación con `caret`, lo cual nos permite buscar automáticamente la mejor combinación de hiperparámetros mediante validación cruzada.


In [ ]:
p_load("ranger")
p_load("caret")


`Caret` nos permite ajustar facilmente los siguientes hiperparámetros

- `mtry`: número de predictores seleccionados aleatoriamente en cada división.
- `splitrule`: criterio de división (en regresión, típicamente `"variance"`).
- `min.node.size`: número mínimo de observaciones en un nodo terminal.
- Falta alguno importante?

In [ ]:
set.seed(123)  # Fijamos semilla para reproducibilidad

tree_ranger_grid <- train(
    log(Sale_Price) ~ Gr_Liv_Area + Bldg_Type + Fence,  # Fórmula del modelo
    data = ames,  # Dataset de entrenamiento
    method = "ranger",  # Usamos el motor ranger para Random Forests
    trControl = fitControl,  # Especificamos los controles de validación cruzada definidos antes
    tuneGrid = expand.grid(   # Definimos la grilla de hiperparámetros a explorar
        mtry = c(1, 2, 3),  # Número de predictores seleccionados al azar en cada división
        splitrule = "variance",  # Regla de partición basada en la reducción de varianza (regresión)
        min.node.size = c(1, 3, 5)  # Tamaño mínimo de nodos terminales
    )#,
    #importance = "permutation"  # Calculamos la importancia de variables por permutación
)


In [ ]:
tree_ranger_grid

Una vez entrenado el modelo, podemos examinar qué tan importantes fueron las variables explicativas en las decisiones del Random Forest. Esto nos da una idea de qué características influyen más en la predicción del precio de las viviendas.

In [ ]:
#varImp(tree_ranger_grid)